In [0]:
#coding:utf-8
import numpy as np
import json
import requests
from requests_oauthlib import OAuth1Session, OAuth1
import datetime
import re
import time

from google.colab import files

## 認証

In [0]:
access_token = 'XXXXXXXXXX
access_token_secret = 'XXXXXXXXXX'
consumer_key = 'XXXXXXXXXX'
consumer_key_secret = 'XXXXXXXXXX'

# タイムライン取得用のURL
url = "https://api.twitter.com/1.1/statuses/user_timeline.json"

#APIの認証
twitter = OAuth1Session(consumer_key, consumer_key_secret, access_token, access_token_secret)

## Np_Ur_ のツイート取得

In [0]:
def normalize_text(text):
    text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
    text = re.sub('RT', "", text)
    text = re.sub('お気に入り', "", text)
    text = re.sub('まとめ', "", text)
    text = re.sub(r'[!-~]', "", text)
    text = re.sub(r'[︰-＠]', "", text)
    text = re.sub('\u3000',"", text)
    text = re.sub('\t', "", text)
    text = re.sub('\n', "", text)
    text = text.strip()
    return text

In [0]:
# パラメータの定義
params = {'screen_name': 'Np_Ur_',
          'exclude_replies':True,
          'include_rts':False,
          'count':200
         }

f_out = open('np_ur_.tsv','w')

for _ in range(20):
    res = twitter.get(url, params = params)

    if res.status_code == 200:

        timeline = json.loads(res.text)
        if len(timeline) == 0:
            break
 
        # 各ツイートの本文を表示
        for i in range(len(timeline)):
            #print(len(timeline[i]['text']))
            f_out.write(normalize_text(timeline[i]['text']) +  '\t' + "0" + '\n')
        
        #  一番最後のツイートIDをパラメータmax_idに追加 
        params['max_id'] = timeline[len(timeline) - 1]['id'] - 1

f_out.close()

In [0]:
files.download('np_ur_.tsv')

## lucky_CandR のツイート取得

In [0]:
# パラメータの定義
params = {'screen_name':'lucky_CandR',
          'exclude_replies':True,
          'include_rts':False,
          'count':200
         }

f_out = open('lucky_CandR.tsv','w')

for _ in range(20):
    res = twitter.get(url, params = params)

    if res.status_code == 200:

        timeline = json.loads(res.text)
        if len(timeline) == 0:
            break
 
        # 各ツイートの本文を表示
        for i in range(len(timeline)):
            f_out.write(normalize_text(timeline[i]['text']) +  '\t' + "1" + '\n')
        
        #  一番最後のツイートIDをパラメータmax_idに追加 
        params['max_id'] = timeline[len(timeline) - 1]['id'] - 1

f_out.close()

In [0]:
files.download('lucky_CandR.tsv')

In [0]:
# データ結合
import pandas as pd

tsv_files = ['np_ur_.tsv', 'lucky_CandR.tsv']
list = []

for file in tsv_files:
    list.append(pd.read_csv(file, delimiter='\t', header=None))
df = pd.concat(list, sort=False)

df.to_csv( 'tweets.tsv', sep='\t',index=False)

In [0]:
files.download('tweets.tsv')